In [164]:
from Bio import Phylo

tree2 = Phylo.read("tree2.new", "newick")

tree1 = Phylo.read("tree1.new", "newick")


In [158]:
with open("tree1.new", "r") as f:
    tree_1 = f.read()
    f.close()

class Node:
   def __init__(self, data):
      self.name = None
      self.left = None
      self.right = None
      self.data = data
   def PrintTree(self):
      print(self.data)

class Leaf:
   def __init__(self, data, distance):
      self.data = data
      self.distance = distance
   def PrintTree(self):
      print(self.data)

class Stack:
   def __init__(self, next):
      self.self = self
      self.next = next


root = Node(10)

root.PrintTree()

10


In [283]:
from Bio import Phylo

tree2 = Phylo.read("tree2.new", "newick")

tree1 = Phylo.read("tree1.new", "newick")

def df_numbering(clade, num_dic):
    out = []
    if clade.is_preterminal():
        for i in clade.clades:
            out.append(num_dic[i.name])
        clade.name = out
        return out
    elif clade.is_terminal():
        return num_dic[clade.name]
    else:
        #print(clade, "yyyyy", clade.is_terminal())
        for i in clade.clades:
            df_numbering(i, num_dic)
            if i.is_terminal():
                out.append(num_dic[i.name])
            else:
                for n in i.name:
                    print(n, out)
                out.append(i.name)
        clade.name = out
        return clade.name
        #print(clade.name)
    
    
    


def rfdist(tree_1, tree_2):
    #Root both trees at same leaf

    tree_1.root_with_outgroup("seq1")
    tree_2.root_with_outgroup("seq1")

    #Make a depth-first numbering of leaves in t1

    tree_1.ladderize()
    DFS = tree_1.get_terminals()

    #rename the leaves in t2 by the DF-numbering of t1
    t2_leaves = tree_2.get_terminals()

    df_numbers = {}
    c = 0
    for i in DFS:
        df_numbers[i.name]=c
        c+=1

    #names are updated in trees [name, DF numbering/interval in subtree]
    print(df_numbers)

    for i in tree_2.find_clades():
    #    print(i.name,1)
        i.name = str(df_numbering(i, df_numbers))
    #    print(i.name,2)
    #    print(tree_2)

    print(tree_2)

    


    
        

def all_parents(tree):
    parents = {}
    for clade in tree.find_clades(order="level"):
        for child in clade:
            parents[child] = clade
    return parents
        
rfdist(tree1, tree2)

{'seq1': 0, 'seq8': 1, 'seq6': 2, 'seq5': 3, 'seq4': 4, 'seq9': 5, 'seq3': 6, 'seq2': 7, 'seq10': 8, 'seq7': 9}
Clade
3 []
4 []
Clade
1 [7]
2 [7]
Clade
7 [[3, 4]]
[1, 2] [[3, 4]]
Clade
[3, 4] []
[7, [1, 2]] []
Clade
[[3, 4], [7, [1, 2]]] [6]
5 [6]
Clade
6 []
[[[3, 4], [7, [1, 2]]], 5] []
Clade
8 [[6, [[[3, 4], [7, [1, 2]]], 5]]]
9 [[6, [[[3, 4], [7, [1, 2]]], 5]]]
Clade
[6, [[[3, 4], [7, [1, 2]]], 5]] []
[8, 9] []


TypeError: __str__ returned non-string (type list)

In [269]:
from Bio import Phylo

tree = Phylo.read("tree1.new", "newick")
tree.root_with_outgroup("seq1")
tree.ladderize()
terminals=tree.get_terminals()
nonterminals= tree.get_nonterminals()

tree.clade


def df_numbering(tree):
    out = []
    if tree.is_preterminal():
        for i in tree.clades:
            out.append(i.name)
        return out
    else:
        for i in tree.clades:
            df_numbering(i)
        

for i in tree.find_clades():
    
    print(i.clades)
for clade in tree.find_clades():
    clade.name = "Michael"
#    print(clade)
    pee = 1

p = all_parents(tree)

f = tree.depths(unit_branch_lengths=True)

print(tree)

Phylo.draw_ascii(tree)

[Clade(branch_length=0.0, name='seq1'), Clade(branch_length=0.2016)]
[]
[Clade(branch_length=0.06209), Clade(branch_length=0.01638)]
[Clade(branch_length=0.12843, name='seq8'), Clade(branch_length=0.12375, name='seq6')]
[]
[]
[Clade(branch_length=0.02587), Clade(branch_length=0.02352)]
[Clade(branch_length=0.30207, name='seq5'), Clade(branch_length=0.24793, name='seq4')]
[]
[]
[Clade(branch_length=0.17922, name='seq9'), Clade(branch_length=0.00628)]
[]
[Clade(branch_length=0.20934, name='seq3'), Clade(branch_length=0.00702)]
[]
[Clade(branch_length=0.18258, name='seq2'), Clade(branch_length=0.02441)]
[]
[Clade(branch_length=0.1471, name='seq10'), Clade(branch_length=0.15261, name='seq7')]
[]
[]
Tree(rooted=True, weight=1.0)
    Clade(name='Michael')
        Clade(branch_length=0.0, name='Michael')
        Clade(branch_length=0.2016, name='Michael')
            Clade(branch_length=0.06209, name='Michael')
                Clade(branch_length=0.12843, name='Michael')
                Clade